In [3]:
#Loading modules

import deepchem as dc
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import Features
from rdkit.Chem import Fragments
from rdkit.Chem import rdMolDescriptors
import tensorflow as tf

import pandas as pd

from matplotlib import pyplot as plt

import numpy as np

from sklearn import metrics
from sklearn.metrics import f1_score

import warnings

# Ensemble learning


In [4]:
#First lets take Richard's featurisation method (shortened):

#Loading the datasets
datasets = dc.molnet.load_tox21(featurizer='GraphConv', splitter='random')


tasks, datasets, transformers = datasets



#Transforming dataset into a pandas dataframe
train_dataset, valid_dataset, test_dataset = datasets

train_df = train_dataset.to_dataframe()
test_dataset = test_dataset.to_dataframe()
valid_dataset = valid_dataset.to_dataframe()

#This line concatenates all the dataframes, we will resplit them later with SKLEARN
dataset = pd.concat([train_df, test_dataset, valid_dataset])

dataset

,X,y1,y2,y3,y4,y5,y6,y7,y8,y9,...,w4,w5,w6,w7,w8,w9,w10,w11,w12,ids
0,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.056284,1.147522,1.055366,1.027673,0.000000,1.037877,0.000000,1.191992,1.067100,FCOC(C(F)(F)F)C(F)(F)F
1,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.056284,1.147522,1.055366,1.027673,1.197585,1.037877,1.060569,1.191992,1.067100,ClCCCCl
2,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,1.147522,1.055366,1.027673,1.197585,1.037877,1.060569,1.191992,1.067100,Nc1ccc2ccccc2c1
3,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.056284,1.147522,1.055366,1.027673,1.197585,1.037877,1.060569,1.191992,1.067100,CCCOC(=O)c1ccccc1C(=O)OCCC
4,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.056284,1.147522,1.055366,1.027673,1.197585,1.037877,1.060569,1.191992,1.067100,O=c1nc([O-])n(Cl)c(=O)n1Cl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.056284,1.147522,1.055366,1.027673,1.197585,1.037877,1.060569,1.191992,1.067100,C[C@H](CCC(=O)[O-])[C@H]1CC[C@H]2[C@H]3[C@H](C...
779,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.056284,1.147522,1.055366,1.027673,1.197585,1.037877,1.060569,1.191992,1.067100,O=C(N[C@H](CO)[C@H](O)c1ccc([N+](=O)[O-])cc1)C...
780,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.056284,1.147522,1.055366,1.027673,1.197585,1.037877,1.060569,1.191992,1.067100,O=C1OC(=O)C2CC=CCC12
781,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.000000,0.000000,1.055366,0.000000,6.061118,0.000000,0.000000,6.208556,15.903226,CCCC[Sn](CCCC)(OC(C)=O)OC(C)=O
